In [1]:
import pandas as pd
import numpy as np
import requests

Using the code from the lesson as a guide, create a dataframe named items that has all of the data for items.

In [5]:
base_url = 'https://python.zach.lol'
items_url = 'https://python.zach.lol/api/v1/items'
sales_url = 'https://python.zach.lol/api/v1/sales'
stores_url = 'https://python.zach.lol/api/v1/stores'

def get_data(key, request_url, base_url):
    response = requests.get(request_url)
    data = response.json()
    page = data['payload']['page']
    max_page = data['payload']['max_page']
    df = pd.DataFrame(data['payload'][key])
    while page <= max_page:
        next_page = data['payload']['next_page']
        if next_page is None or next_page == "None":
            break
        response = requests.get(base_url + next_page)
        data = response.json()
        df = pd.concat([df, pd.DataFrame(data['payload'][key])])
        page = data['payload']['page']

    df.reset_index(inplace=True)
    return df

sales = get_data('sales', sales_url, base_url)
sales.shape

,index,item,sale_amount,sale_date,sale_id,store
0,0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [51]:
import pandas as pd
import numpy as np
import requests
import os

base_url = 'https://python.zach.lol'
items_url = 'https://python.zach.lol/api/v1/items'
sales_url = 'https://python.zach.lol/api/v1/sales'
stores_url = 'https://python.zach.lol/api/v1/stores'

def get_zach_data(key, request_url, base_url=base_url):
    '''
    Utlizes rest API to gather data from Zach's python stash
    Keys = items, stores, sales for each their respective data sets
    request_url = items_url, sales_url, or stores_url for their respective data sets
    base_url = base_url and referes to 'https://python.zach.lol'
    '''
    
    response = requests.get(request_url)
    data = response.json()
    page = data['payload']['page']
    max_page = data['payload']['max_page']
        
    file = f'{key}.csv'

    if os.path.isfile(file):
        df = pd.read_csv(f'{key}.csv')
        return df
    
    else:
        df = pd.DataFrame(data['payload'][key])
        while page <= max_page:
            next_page = data['payload']['next_page']
            if next_page is None or next_page == "None":
                break
            response = requests.get(base_url + next_page)
            data = response.json()
            df = pd.concat([df, pd.DataFrame(data['payload'][key])])
            page = data['payload']['page']

        df.reset_index(inplace=True, drop=True)
        df.to_csv(str(key) + '.csv', index=False)
        return df
    
    return df

def all_zach_data():
    '''
    Once you've loaded all zach's into CSV using the get_zach_data function you can run this function to stitch the individual 
    data frames together
    '''
    sales = pd.read_csv('sales.csv')
    items = pd.read_csv('items.csv')
    stores = pd.read_csv('stores.csv')
    full = sales.merge(store, left_on='store', right_on='store_id')
    full = full.merge(items, left_on='item', right_on='item_id')
    return full
    
def get_de_electric():
    '''
    Load the Open Power Systems Data for Germany by reading the CSV url into the a DF and CSV (de_electric) if applicable
    '''
    file = 'de_electric.csv'

    if os.path.isfile(file):
        df = pd.read_csv('de_electric.csv')
        return df
    
    else:
        df = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')
        return df

Combine the data from your three separate dataframes into one large dataframe.

In [46]:
full = all_zach_data()
full

,item,sale_amount,sale_date,sale_id,store,store_address,store_city,store_id,store_state,store_zipcode,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912995,50,63.0,"Wed, 27 Dec 2017 00:00:00 GMT",912996,10,8503 NW Military Hwy,San Antonio,10,TX,78231,Choice,50,Choice Organic Teas Black Tea Classic Black - ...,5.20,47445919221,47445919221
912996,50,59.0,"Thu, 28 Dec 2017 00:00:00 GMT",912997,10,8503 NW Military Hwy,San Antonio,10,TX,78231,Choice,50,Choice Organic Teas Black Tea Classic Black - ...,5.20,47445919221,47445919221
912997,50,74.0,"Fri, 29 Dec 2017 00:00:00 GMT",912998,10,8503 NW Military Hwy,San Antonio,10,TX,78231,Choice,50,Choice Organic Teas Black Tea Classic Black - ...,5.20,47445919221,47445919221
912998,50,62.0,"Sat, 30 Dec 2017 00:00:00 GMT",912999,10,8503 NW Military Hwy,San Antonio,10,TX,78231,Choice,50,Choice Organic Teas Black Tea Classic Black - ...,5.20,47445919221,47445919221


DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes='_x', '_y', copy=True, indicator=False, validate=None)[source]

In [39]:
items

,index,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036
5,5,Sally Hansen,6,Sally Hansen Nail Color Magnetic 903 Silver El...,6.93,74170388732,74170388732
6,6,Twinings Of London,7,Twinings Of London Classics Lady Grey Tea - 20 Ct,9.64,70177154004,70177154004
7,7,Lea & Perrins,8,Lea & Perrins Marinade In-a-bag Cracked Pepper...,1.68,51600080015,51600080015
8,8,Van De Kamps,9,Van De Kamps Fillets Beer Battered - 10 Ct,1.79,19600923015,19600923015
9,9,Ahold,10,Ahold Cocoa Almonds,3.17,688267141676,688267141676


In [34]:
store

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218
5,1015 S WW White Rd,San Antonio,6,TX,78220
6,12018 Perrin Beitel Rd,San Antonio,7,TX,78217
7,15000 San Pedro Ave,San Antonio,8,TX,78232
8,735 SW Military Dr,San Antonio,9,TX,78221
9,8503 NW Military Hwy,San Antonio,10,TX,78231


Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.

In [52]:
df = get_de_electric()
df

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.18400,NaN,NaN,NaN
1,2006-01-02,1380.52100,NaN,NaN,NaN
2,2006-01-03,1442.53300,NaN,NaN,NaN
3,2006-01-04,1457.21700,NaN,NaN,NaN
4,2006-01-05,1477.13100,NaN,NaN,NaN
...,...,...,...,...,...
4378,2017-12-27,1263.94091,394.507,16.530,411.037
4379,2017-12-28,1299.86398,506.424,14.162,520.586
4380,2017-12-29,1295.08753,584.277,29.854,614.131
4381,2017-12-30,1215.44897,721.247,7.467,728.714
